In [1]:
%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os

load_dotenv()

chat = AzureChatOpenAI(
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSION'),
    api_key=os.getenv('AZURE_OPENAI_KEY')
)    

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

template = "Tell me a joke about {topic}."

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is Carl."),
    ("human", template),
])

# Use a string parser and to avoid need to do result.content
#  So this add the prompt --> chat --> end you parse the output
chain = prompt_template | chat | StrOutputParser()

result = await chain.ainvoke({"topic": "cat"})

result

"Sure, here's a cat joke for you:\n\nWhy did the cat sit on the computer?\n\nBecause it wanted to keep an eye on the mouse!"

In [ ]:
from langchain.schema.runnable import RunnableLambda

prompt_template = ChatPromptTemplate([
    ("system", "You are a comedian who tells jokes about {topic}"),
    ("human", "Tell me {joke_count} jokes."),
])

uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Words count: {len(x.split())}\n{x}")

chain = prompt_template | chat | StrOutputParser() | uppercase_output | count_words

result = await chain.ainvoke({"topic": "cat", "joke_count": 3})

result

Chain parallel

In [7]:
from langchain.schema.runnable import RunnableLambda, RunnableParallel

prompt_template = ChatPromptTemplate([
    ("system", "You are an expert product reviewer"),
    ("human", "List the main features of the product {product_name}.")
])

def analyze_pros(features):
    pros_template = ChatPromptTemplate([
        ("system", "You are an expert product reviewer"),
        ("human", "Given these features: {features}, list the pros of these features.")
    ])
    return pros_template.format_prompt(features=features)

def analyze_cons(features):
    cons_template = ChatPromptTemplate([
        ("system", "You are an expert product reviewer"),
        ("human", "Given these features: {features}, list the cons of these features.")
    ])
    return cons_template.format_prompt(features=features)

def combine_pros_cons(pros, cons):
    return f"Pros:\n {pros}\n\nCons:\n {cons}"

pros_branch = (
    RunnableLambda(lambda x: analyze_pros(x)) | chat | StrOutputParser()
)

cons_branch = (
    RunnableLambda(lambda x: analyze_cons(x)) | chat | StrOutputParser()
)

chain = (
    prompt_template
    | chat
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch, "cons": cons_branch})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"])) 
)

result = await chain.ainvoke({"product_name": "iPhone 13"})

result

"Pros:\n Pros of the iPhone 13's main features:\n\n1. Display: The Super Retina XDR display offers excellent color accuracy, brightness, and HDR support, providing a visually stunning viewing experience.\n\n2. A15 Bionic Chip: The latest A15 Bionic chip delivers improved performance, making the iPhone 13 faster and more efficient for tasks like gaming and AR.\n\n3. Camera System: The dual-camera system with advanced computational photography features allows for high-quality photos and videos, even in low-light conditions. The support for 4K Dolby Vision HDR recording adds to the overall camera capabilities.\n\n4. Battery Life: The longer battery life of the iPhone 13 means users can enjoy extended usage without worrying about running out of power quickly.\n\n5. 5G Connectivity: The 5G support enables faster download and streaming speeds, enhancing the overall internet experience on the iPhone 13.\n\n6. Storage Options: The various storage configurations provide ample space for storing 

Chain Branching

In [8]:
from langchain.schema.runnable import RunnableBranch

positive_feedback_template = ChatPromptTemplate([
    ("system", "You are an helpfull assistant"),
    ("human", "Generate a thank you note for this positive feedback: {feedback}.")
])

negative_feedback_template = ChatPromptTemplate([
    ("system", "You are an helpfull assistant"),
    ("human", "Generate a reponse addressing the negative feedback: {feedback}.")
])

neutral_feedback_template = ChatPromptTemplate([
    ("system", "You are an helpfull assistant"),
    ("human", "Generate a request for more details for this neutral feedback: {feedback}.")
])

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

# Define the feedback classification template
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)

branches = RunnableBranch(
    # This is like IF statement, if the condition is true, it will execute the chain
    # otherwise does the others
    (
        lambda x: "positive" in x,
        positive_feedback_template | chat | StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | chat | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | chat | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | chat | StrOutputParser()
)

# Create the classification chain
classification_chain = classification_template | chat | StrOutputParser()

# Combine classification and response generation into one chain
chain = classification_chain | branches

review = "The product is terrible. It broke after just one use and the quality is very poor."
result = await chain.ainvoke({"feedback": review})

result

'Dear valued customer,\n\nThank you for taking the time to provide us with your feedback. We sincerely apologize for any negative experience you may have had with our product/service. \n\nAt [Company Name], we strive to ensure the utmost satisfaction of our customers, and we deeply regret falling short of your expectations. We understand the importance of addressing any concerns you may have, and we are committed to resolving the issue to your satisfaction.\n\nWe would greatly appreciate it if you could provide us with more details about your specific experience so that we can investigate the matter further. Our team is dedicated to improving and delivering the best possible service, and your feedback is invaluable in helping us achieve that goal.\n\nPlease feel free to reach out to our customer support team at [contact details] to discuss this matter further. We genuinely value your opinion and are eager to work towards a resolution that meets your needs.\n\nOnce again, we apologize f